In [ ]:
import numpy as np
from typing import Optional, Tuple, List, Dict, Union
from enum import Enum
import warnings
from dataclasses import dataclass

class OptimizationStatus(Enum):
    OPTIMAL = "optimal"
    UNBOUNDED = "unbounded"
    INFEASIBLE = "infeasible"
    MAX_ITERATIONS = "max_iterations_reached"
    NUMERICAL_ERROR = "numerical_error"

class PivotRule(Enum):
    DANTZIG = "dantzig"  # Most negative reduced cost
    BLAND = "bland"      # Anti-cycling rule
    STEEPEST_EDGE = "steepest_edge"  # Steepest edge rule

@dataclass
class SimplexResult:
    status: OptimizationStatus
    x: Optional[np.ndarray] = None
    objective_value: Optional[float] = None
    iterations: int = 0
    basic_variables: Optional[List[int]] = None
    reduced_costs: Optional[np.ndarray] = None
    shadow_prices: Optional[np.ndarray] = None

class SimplexSolver:
    """
    State-of-the-art Simplex solver supporting both primal and dual simplex methods.
    
    Features:
    - Primal and dual simplex algorithms
    - Multiple pivot rules (Dantzig, Bland's anti-cycling, Steepest Edge)
    - Numerical stability improvements
    - Phase I method for finding initial feasible solution
    - Degeneracy handling
    - Comprehensive result reporting
    """
    
    def __init__(self, 
                 pivot_rule: PivotRule = PivotRule.DANTZIG,
                 tolerance: float = 1e-10,
                 max_iterations: int = 10000,
                 verbose: bool = False):
        self.pivot_rule = pivot_rule
        self.tol = tolerance
        self.max_iterations = max_iterations
        self.verbose = verbose
        
    def solve(self, 
              c: np.ndarray, 
              A: np.ndarray, 
              b: np.ndarray,
              method: str = "primal") -> SimplexResult:
        """
        Solve linear program: min c^T x subject to Ax = b, x >= 0
        
        Args:
            c: Objective coefficients (n,)
            A: Constraint matrix (m, n)
            b: RHS vector (m,)
            method: "primal" or "dual"
            
        Returns:
            SimplexResult object with solution details
        """
        # Input validation and preprocessing
        c, A, b = self._preprocess_inputs(c, A, b)
        
        if method == "primal":
            return self._solve_primal(c, A, b)
        elif method == "dual":
            return self._solve_dual(c, A, b)
        else:
            raise ValueError("Method must be 'primal' or 'dual'")
    
    def _preprocess_inputs(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Validate and preprocess inputs"""
        c = np.asarray(c, dtype=np.float64)
        A = np.asarray(A, dtype=np.float64)
        b = np.asarray(b, dtype=np.float64)
        
        if A.ndim != 2:
            raise ValueError("A must be 2-dimensional")
        if c.ndim != 1:
            raise ValueError("c must be 1-dimensional")
        if b.ndim != 1:
            raise ValueError("b must be 1-dimensional")
        if A.shape[1] != len(c):
            raise ValueError("Number of columns in A must match length of c")
        if A.shape[0] != len(b):
            raise ValueError("Number of rows in A must match length of b")
            
        return c, A, b
    
    def _solve_primal(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> SimplexResult:
        """Solve using primal simplex method"""
        m, n = A.shape
        
        # Phase I: Find initial basic feasible solution
        phase1_result = self._phase1(A, b)
        if phase1_result.status != OptimizationStatus.OPTIMAL:
            return phase1_result
            
        # Extract Phase I solution
        tableau = phase1_result.tableau
        basic_vars = phase1_result.basic_variables
        
        # Phase II: Optimize original objective
        return self._phase2_primal(c, tableau, basic_vars)
    
    def _solve_dual(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> SimplexResult:
        """Solve using dual simplex method"""
        m, n = A.shape
        
        # For dual simplex to work, we need a dual feasible starting point
        # This typically means starting with reduced costs >= 0 for minimization
        # But we may have primal infeasibility (negative RHS values)
        
        # Set up initial tableau with slack variables
        tableau, basic_vars = self._setup_dual_tableau(c, A, b)
        
        # Check if we have negative RHS values (primal infeasibility)
        rhs = tableau[:-1, -1]
        if np.all(rhs >= -self.tol):
            # Already primal feasible, just solve normally with primal simplex
            return self._primal_simplex_iterations(tableau, basic_vars, is_phase1=False)
        
        # We have primal infeasibility, proceed with dual simplex
        return self._dual_simplex_iterations(tableau, basic_vars)
    
    def _phase1(self, A: np.ndarray, b: np.ndarray) -> SimplexResult:
        """Phase I: Find initial basic feasible solution"""
        m, n = A.shape
        
        # Check if we can immediately detect infeasibility
        # If any constraint has all non-negative coefficients but negative RHS, it's infeasible
        for i in range(m):
            if b[i] < -self.tol and np.all(A[i, :] >= -self.tol):
                return SimplexResult(status=OptimizationStatus.INFEASIBLE)
        
        # Check if any RHS is negative - need artificial variables
        needs_artificial = np.any(b < -self.tol)
        
        if not needs_artificial and np.all(b >= -self.tol):
            # Already have a basic feasible solution with slack variables
            # Create tableau with slack variables as basic
            tableau = np.zeros((m + 1, n + m + 1))
            tableau[:m, :n] = A
            tableau[:m, n:n+m] = np.eye(m)
            tableau[:m, -1] = b
            # Objective row will be set up in Phase II
            
            basic_vars = list(range(n, n + m))  # Slack variables are basic
            
            result = SimplexResult(status=OptimizationStatus.OPTIMAL, objective_value=0)
            result.tableau = tableau
            result.basic_variables = basic_vars
            return result
        
        # Need artificial variables for negative RHS or to find initial feasible solution
        # Convert negative RHS by multiplying constraints by -1
        A_phase1 = A.copy()
        b_phase1 = b.copy()
        
        for i in range(m):
            if b_phase1[i] < -self.tol:
                A_phase1[i, :] *= -1
                b_phase1[i] *= -1
        
        # Now check again for obvious infeasibility after sign flipping
        for i in range(m):
            if b_phase1[i] < -self.tol and np.all(A_phase1[i, :] >= -self.tol):
                return SimplexResult(status=OptimizationStatus.INFEASIBLE)
        
        # Add artificial variables
        # min sum(artificial vars) subject to Ax + I*art_vars = b
        tableau = np.zeros((m + 1, n + m + m + 1))  # Original vars + slack + artificial vars + RHS
        tableau[:m, :n] = A_phase1
        tableau[:m, n:n+m] = np.eye(m)  # Slack variables
        tableau[:m, n+m:n+m+m] = np.eye(m)  # Artificial variables
        tableau[:m, -1] = b_phase1
        tableau[-1, n+m:n+m+m] = 1  # Objective: minimize sum of artificial variables
        
        # Basic variables are the artificial variables initially
        basic_vars = list(range(n + m, n + m + m))
        
        # Eliminate artificial variables from objective row (make reduced costs zero for basic vars)
        for i in range(m):
            tableau[-1, :] -= tableau[i, :]
        
        # Solve Phase I
        result = self._primal_simplex_iterations(tableau, basic_vars, is_phase1=True)
        
        if result.status != OptimizationStatus.OPTIMAL:
            return result
        
        # Check if Phase I solution is feasible for original problem
        if abs(result.objective_value) > self.tol:
            return SimplexResult(status=OptimizationStatus.INFEASIBLE)
        
        # Remove artificial variables from tableau and keep only original + slack variables
        tableau_phase2 = np.zeros((m + 1, n + m + 1))
        tableau_phase2[:m, :n+m] = tableau[:m, :n+m]  # Original + slack variables
        tableau_phase2[:m, -1] = tableau[:m, -1]  # RHS
        
        # Update basic variables (remove artificial variable indices)
        basic_vars_phase2 = []
        for i, var in enumerate(basic_vars):
            if var < n + m:  # Keep only original and slack variables
                basic_vars_phase2.append(var)
            else:
                # This basic variable was artificial, need to find replacement
                # Look for a non-artificial variable in this row that can be made basic
                found_replacement = False
                for j in range(n + m):
                    if abs(tableau[i, j]) > self.tol:
                        basic_vars_phase2.append(j)
                        found_replacement = True
                        break
                if not found_replacement:
                    # This shouldn't happen in a properly implemented Phase I
                    basic_vars_phase2.append(0)  # Fallback
        
        result.tableau = tableau_phase2
        result.basic_variables = basic_vars_phase2
        return result
    
    def _phase2_primal(self, c: np.ndarray, tableau: np.ndarray, basic_vars: List[int]) -> SimplexResult:
        """Phase II: Optimize original objective"""
        m, n_plus_1 = tableau.shape
        n_total = n_plus_1 - 1  # Total variables including slack
        n_orig = len(c)  # Original variables only
        
        # Extend objective vector to include slack variables (coefficient 0)
        c_extended = np.zeros(n_total)
        c_extended[:n_orig] = c
        
        # Set up objective row with extended costs
        tableau[-1, :n_total] = c_extended
        tableau[-1, -1] = 0
        
        # Eliminate basic variables from objective row
        for i, var in enumerate(basic_vars):
            if var < n_total and abs(tableau[-1, var]) > self.tol:
                factor = tableau[-1, var]
                tableau[-1, :] -= factor * tableau[i, :]
        
        return self._primal_simplex_iterations(tableau, basic_vars, is_phase1=False)
    
    def _primal_simplex_iterations(self, tableau: np.ndarray, basic_vars: List[int], is_phase1: bool = False) -> SimplexResult:
        """Main primal simplex iteration loop"""
        iterations = 0
        m, n_plus_1 = tableau.shape
        n = n_plus_1 - 1
        
        while iterations < self.max_iterations:
            # Check optimality (all reduced costs non-negative for minimization)
            reduced_costs = tableau[-1, :n]
            
            if self.verbose:
                print(f"Iteration {iterations}: Reduced costs = {reduced_costs}")
                print(f"RHS = {tableau[:-1, -1]}")
                print(f"Basic vars = {basic_vars}")
                print(f"Current objective = {tableau[-1, -1]}")
            
            # For minimization: optimal when all reduced costs >= 0
            if np.all(reduced_costs >= -self.tol):
                # Optimal solution found
                x = np.zeros(n)
                for i, var in enumerate(basic_vars):
                    if var < n and tableau[i, -1] >= -self.tol:
                        x[var] = tableau[i, -1]
                
                # For Phase I, objective value is the sum of artificial variables (should be 0)
                # For Phase II, objective value is the negative of the tableau value (since we minimize)
                obj_value = tableau[-1, -1] if is_phase1 else -tableau[-1, -1]
                
                return SimplexResult(
                    status=OptimizationStatus.OPTIMAL,
                    x=x,
                    objective_value=obj_value,
                    iterations=iterations,
                    basic_variables=basic_vars.copy(),
                    reduced_costs=reduced_costs.copy()
                )
            
            # Choose entering variable (most negative reduced cost for minimization)
            entering_var = self._choose_entering_variable(reduced_costs)
            
            if self.verbose:
                print(f"Entering variable: {entering_var}")
            
            # Choose leaving variable (ratio test)
            leaving_idx = self._ratio_test(tableau, entering_var)
            
            if leaving_idx == -1:
                return SimplexResult(status=OptimizationStatus.UNBOUNDED)
            
            if self.verbose:
                print(f"Leaving variable: {basic_vars[leaving_idx]} (row {leaving_idx})")
            
            # Pivot operation
            self._pivot(tableau, leaving_idx, entering_var)
            basic_vars[leaving_idx] = entering_var
            
            iterations += 1
            
            if self.verbose and iterations % 10 == 0:
                print(f"Iteration {iterations}, objective: {-tableau[-1, -1]}")
        
        return SimplexResult(status=OptimizationStatus.MAX_ITERATIONS, iterations=iterations)
    
    def _dual_simplex_iterations(self, tableau: np.ndarray, basic_vars: List[int]) -> SimplexResult:
        """Main dual simplex iteration loop"""
        iterations = 0
        m, n_plus_1 = tableau.shape
        n = n_plus_1 - 1
        
        while iterations < self.max_iterations:
            # Check dual feasibility (reduced costs should be >= 0 for minimization)
            # and primal feasibility (all RHS non-negative)
            rhs = tableau[:-1, -1]
            reduced_costs = tableau[-1, :n]
            
            if self.verbose:
                print(f"Dual iteration {iterations}: RHS = {rhs}")
                print(f"Reduced costs = {reduced_costs}")
                print(f"Basic vars = {basic_vars}")
            
            # Check primal feasibility (all RHS >= 0)
            if np.all(rhs >= -self.tol):
                # Optimal solution found
                x = np.zeros(n)
                for i, var in enumerate(basic_vars):
                    if var < n:
                        x[var] = max(0, tableau[i, -1])
                
                return SimplexResult(
                    status=OptimizationStatus.OPTIMAL,
                    x=x,
                    objective_value=-tableau[-1, -1],  # Negative because we're minimizing
                    iterations=iterations,
                    basic_variables=basic_vars.copy(),
                    reduced_costs=reduced_costs.copy()
                )
            
            # Choose leaving variable (most negative RHS)
            leaving_idx = np.argmin(rhs)
            if rhs[leaving_idx] >= -self.tol:
                break
            
            if self.verbose:
                print(f"Leaving variable: {basic_vars[leaving_idx]} (row {leaving_idx})")
            
            # Choose entering variable (dual ratio test)
            entering_var = self._dual_ratio_test(tableau, leaving_idx)
            
            if entering_var == -1:
                return SimplexResult(status=OptimizationStatus.INFEASIBLE)
            
            if self.verbose:
                print(f"Entering variable: {entering_var}")
            
            # Pivot operation
            self._pivot(tableau, leaving_idx, entering_var)
            basic_vars[leaving_idx] = entering_var
            
            iterations += 1
            
            if self.verbose and iterations % 100 == 0:
                print(f"Dual iteration {iterations}, objective: {-tableau[-1, -1]}")
        
        return SimplexResult(status=OptimizationStatus.MAX_ITERATIONS, iterations=iterations)
    
    def _setup_dual_tableau(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, List[int]]:
        """Set up initial tableau for dual simplex"""
        m, n = A.shape
        
        # For dual simplex, we need dual feasibility but not necessarily primal feasibility
        # Standard form: min c^T x s.t. Ax = b, x >= 0
        # We add slack/surplus variables to make it Ax <= b first, then convert
        
        # The tableau setup: we want to start with a dual feasible solution
        # This means reduced costs should be non-positive for a minimization problem
        tableau = np.zeros((m + 1, n + m + 1))
        tableau[:m, :n] = A
        tableau[:m, n:n+m] = np.eye(m)  # Slack variables
        tableau[:m, -1] = b
        
        # Set up objective row - coefficients should be set up for dual feasibility
        # For the original variables, use the given costs
        tableau[-1, :n] = c
        # For slack variables, cost is 0 (already initialized)
        
        # Basic variables start as slack variables
        basic_vars = list(range(n, n + m))
        
        return tableau, basic_vars
    
    def _choose_entering_variable(self, reduced_costs: np.ndarray) -> int:
        """Choose entering variable based on pivot rule"""
        if self.pivot_rule == PivotRule.DANTZIG:
            # Most negative reduced cost
            return np.argmin(reduced_costs)
        elif self.pivot_rule == PivotRule.BLAND:
            # First negative reduced cost (anti-cycling)
            for i, cost in enumerate(reduced_costs):
                if cost < -self.tol:
                    return i
            return -1
        else:  # STEEPEST_EDGE
            # Simple steepest edge approximation
            return np.argmin(reduced_costs)
    
    def _ratio_test(self, tableau: np.ndarray, entering_var: int) -> int:
        """Minimum ratio test for choosing leaving variable"""
        m = tableau.shape[0] - 1
        pivot_col = tableau[:-1, entering_var]
        rhs = tableau[:-1, -1]
        
        min_ratio = float('inf')
        leaving_idx = -1
        
        for i in range(m):
            if pivot_col[i] > self.tol:  # Positive pivot element
                ratio = rhs[i] / pivot_col[i]
                if ratio < min_ratio:
                    min_ratio = ratio
                    leaving_idx = i
        
        return leaving_idx
    
    def _dual_ratio_test(self, tableau: np.ndarray, leaving_idx: int) -> int:
        """Dual ratio test for choosing entering variable in dual simplex"""
        n = tableau.shape[1] - 1 - tableau.shape[0] + 1  # Original variables
        pivot_row = tableau[leaving_idx, :n]
        reduced_costs = tableau[-1, :n]
        
        min_ratio = float('inf')
        entering_var = -1
        
        for j in range(n):
            if pivot_row[j] < -self.tol:  # Negative pivot element
                ratio = reduced_costs[j] / pivot_row[j]
                if ratio < min_ratio:
                    min_ratio = ratio
                    entering_var = j
        
        return entering_var
    
    def _pivot(self, tableau: np.ndarray, pivot_row: int, pivot_col: int):
        """Perform pivot operation"""
        pivot_element = tableau[pivot_row, pivot_col]
        
        if abs(pivot_element) < self.tol:
            raise ValueError("Pivot element too small - numerical instability")
        
        # Scale pivot row
        tableau[pivot_row, :] /= pivot_element
        
        # Eliminate other entries in pivot column
        for i in range(tableau.shape[0]):
            if i != pivot_row and abs(tableau[i, pivot_col]) > self.tol:
                factor = tableau[i, pivot_col]
                tableau[i, :] -= factor * tableau[pivot_row, :]

def solve_lp(c: np.ndarray, 
             A: np.ndarray, 
             b: np.ndarray,
             method: str = "primal",
             pivot_rule: str = "dantzig",
             verbose: bool = False) -> SimplexResult:
    """
    Convenience function to solve linear program.
    
    Args:
        c: Objective coefficients
        A: Constraint matrix  
        b: RHS vector
        method: "primal" or "dual"
        pivot_rule: "dantzig", "bland", or "steepest_edge"
        verbose: Print iteration info
        
    Returns:
        SimplexResult with solution
    """
    pivot_rule_enum = {
        "dantzig": PivotRule.DANTZIG,
        "bland": PivotRule.BLAND, 
        "steepest_edge": PivotRule.STEEPEST_EDGE
    }[pivot_rule.lower()]
    
    solver = SimplexSolver(pivot_rule=pivot_rule_enum, verbose=verbose)
    return solver.solve(c, A, b, method=method)

class RevisedSimplexSolver:
    """
    Revised Simplex Method implementation for large-scale linear programming.
    
    The revised simplex method maintains the basis inverse B^{-1} explicitly
    and computes tableau columns on-demand, making it much more memory-efficient
    and numerically stable for large problems.
    
    Features:
    - Explicit basis inverse maintenance
    - LU factorization with partial pivoting
    - Eta matrix updates for numerical stability
    - FTRAN/BTRAN operations for efficient column generation
    - Basis factorization refresh to prevent numerical degradation
    """
    
    def __init__(self, 
                 pivot_rule: PivotRule = PivotRule.DANTZIG,
                 tolerance: float = 1e-10,
                 max_iterations: int = 10000,
                 refactor_frequency: int = 50,
                 verbose: bool = False):
        self.pivot_rule = pivot_rule
        self.tol = tolerance
        self.max_iterations = max_iterations
        self.refactor_frequency = refactor_frequency  # How often to refresh factorization
        self.verbose = verbose
        
    def solve(self, 
              c: np.ndarray, 
              A: np.ndarray, 
              b: np.ndarray) -> SimplexResult:
        """
        Solve LP using revised simplex method: min c^T x subject to Ax = b, x >= 0
        """
        c, A, b = self._preprocess_inputs(c, A, b)
        return self._solve_revised(c, A, b)
    
    def _preprocess_inputs(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Validate and preprocess inputs"""
        c = np.asarray(c, dtype=np.float64)
        A = np.asarray(A, dtype=np.float64)
        b = np.asarray(b, dtype=np.float64)
        
        if A.ndim != 2:
            raise ValueError("A must be 2-dimensional")
        if c.ndim != 1:
            raise ValueError("c must be 1-dimensional")
        if b.ndim != 1:
            raise ValueError("b must be 1-dimensional")
        if A.shape[1] != len(c):
            raise ValueError("Number of columns in A must match length of c")
        if A.shape[0] != len(b):
            raise ValueError("Number of rows in A must match length of b")
            
        return c, A, b
    
    def _solve_revised(self, c: np.ndarray, A: np.ndarray, b: np.ndarray) -> SimplexResult:
        """Main revised simplex algorithm"""
        m, n = A.shape
        
        # Phase I: Find initial basic feasible solution
        phase1_result = self._revised_phase1(A, b)
        if phase1_result.status != OptimizationStatus.OPTIMAL:
            return phase1_result
        
        # Extract Phase I results
        basic_vars = phase1_result.basic_variables
        
        # Phase II: Optimize with revised simplex
        return self._revised_phase2(c, A, b, basic_vars)
    
    def _revised_phase1(self, A: np.ndarray, b: np.ndarray) -> SimplexResult:
        """Phase I using revised simplex method"""
        m, n = A.shape
        
        # Check for obvious infeasibility
        for i in range(m):
            if b[i] < -self.tol and np.all(A[i, :] >= -self.tol):
                return SimplexResult(status=OptimizationStatus.INFEASIBLE)
        
        # Handle negative RHS
        A_phase1 = A.copy()
        b_phase1 = b.copy()
        
        for i in range(m):
            if b_phase1[i] < -self.tol:
                A_phase1[i, :] *= -1
                b_phase1[i] *= -1
        
        # If all RHS are non-negative, we can start with slack variables
        if np.all(b_phase1 >= -self.tol):
            # Assume we have slack variables as the last m columns
            basic_vars = list(range(n - m, n))
            result = SimplexResult(status=OptimizationStatus.OPTIMAL)
            result.basic_variables = basic_vars
            return result
        
        # Need artificial variables - use standard Phase I approach
        # Add artificial variables: A_aug = [A I]
        A_aug = np.hstack([A_phase1, np.eye(m)])
        c_phase1 = np.concatenate([np.zeros(n), np.ones(m)])
        
        # Initial basic variables are artificial variables
        basic_vars = list(range(n, n + m))
        
        # Solve Phase I problem
        result = self._revised_iterations(c_phase1, A_aug, b_phase1, basic_vars, is_phase1=True)
        
        if result.status != OptimizationStatus.OPTIMAL:
            return result
        
        # Check feasibility
        if abs(result.objective_value) > self.tol:
            return SimplexResult(status=OptimizationStatus.INFEASIBLE)
        
        # Remove artificial variables from basic set
        basic_vars_phase2 = [var for var in result.basic_variables if var < n]
        
        # If we have fewer than m basic variables, we need to find more
        while len(basic_vars_phase2) < m:
            # This is a degenerate case - add a slack variable
            basic_vars_phase2.append(n - m + len(basic_vars_phase2))
        
        result.basic_variables = basic_vars_phase2
        return result
    
    def _revised_phase2(self, c: np.ndarray, A: np.ndarray, b: np.ndarray, basic_vars: List[int]) -> SimplexResult:
        """Phase II using revised simplex method"""
        return self._revised_iterations(c, A, b, basic_vars, is_phase1=False)
    
    def _revised_iterations(self, c: np.ndarray, A: np.ndarray, b: np.ndarray, 
                           basic_vars: List[int], is_phase1: bool = False) -> SimplexResult:
        """Main revised simplex iteration loop"""
        m, n = A.shape
        iterations = 0
        refactor_count = 0
        
        # Initialize basis matrix and its inverse
        B = A[:, basic_vars]
        
        try:
            B_inv = np.linalg.inv(B)
        except np.linalg.LinAlgError:
            return SimplexResult(status=OptimizationStatus.NUMERICAL_ERROR)
        
        # LU factorization for numerical stability
        from scipy.linalg import lu_factor, lu_solve
        
        while iterations < self.max_iterations:
            # Refresh factorization periodically for numerical stability
            if iterations % self.refactor_frequency == 0 and iterations > 0:
                B = A[:, basic_vars]
                try:
                    B_inv = np.linalg.inv(B)
                    refactor_count += 1
                except np.linalg.LinAlgError:
                    return SimplexResult(status=OptimizationStatus.NUMERICAL_ERROR)
            
            # Step 1: Compute basic solution
            # x_B = B^{-1} * b
            x_B = B_inv @ b
            
            # Check primal feasibility
            if np.any(x_B < -self.tol):
                return SimplexResult(status=OptimizationStatus.NUMERICAL_ERROR)
            
            # Step 2: Compute dual solution (shadow prices)
            # π = c_B^T * B^{-1}
            c_B = c[basic_vars]
            pi = c_B @ B_inv
            
            # Step 3: Compute reduced costs for all non-basic variables
            # c_j - π * A_j for all non-basic j
            reduced_costs = np.zeros(n)
            entering_var = -1
            min_reduced_cost = 0
            
            for j in range(n):
                if j not in basic_vars:
                    reduced_cost = c[j] - pi @ A[:, j]
                    reduced_costs[j] = reduced_cost
                    
                    # Check optimality condition (minimization: all reduced costs >= 0)
                    if reduced_cost < min_reduced_cost - self.tol:
                        min_reduced_cost = reduced_cost
                        entering_var = j
            
            if self.verbose:
                print(f"Iteration {iterations}: Min reduced cost = {min_reduced_cost}")
                print(f"Basic variables: {basic_vars}")
                print(f"Basic solution: {x_B}")
            
            # Check optimality
            if entering_var == -1:
                # Optimal solution found
                x = np.zeros(n)
                for i, var in enumerate(basic_vars):
                    x[var] = x_B[i]
                
                obj_value = c @ x if not is_phase1 else sum(x[var] for var in basic_vars if var >= n - m)
                
                return SimplexResult(
                    status=OptimizationStatus.OPTIMAL,
                    x=x,
                    objective_value=obj_value,
                    iterations=iterations,
                    basic_variables=basic_vars.copy(),
                    reduced_costs=reduced_costs.copy(),
                    shadow_prices=pi.copy()
                )
            
            # Step 4: Compute entering column in basis coordinates
            # d = B^{-1} * A_entering (FTRAN operation)
            A_entering = A[:, entering_var]
            d = B_inv @ A_entering
            
            if self.verbose:
                print(f"Entering variable: {entering_var}")
                print(f"Direction vector: {d}")
            
            # Step 5: Ratio test to find leaving variable
            leaving_idx = -1
            min_ratio = float('inf')
            
            for i in range(m):
                if d[i] > self.tol:  # Only consider positive direction components
                    ratio = x_B[i] / d[i]
                    if ratio < min_ratio:
                        min_ratio = ratio
                        leaving_idx = i
            
            # Check for unboundedness
            if leaving_idx == -1:
                return SimplexResult(status=OptimizationStatus.UNBOUNDED)
            
            leaving_var = basic_vars[leaving_idx]
            
            if self.verbose:
                print(f"Leaving variable: {leaving_var} (index {leaving_idx})")
                print(f"Ratio: {min_ratio}")
            
            # Step 6: Update basis inverse using Sherman-Morrison formula
            # B_inv_new = B_inv - (B_inv * A_entering * e_leaving^T * B_inv) / pivot
            pivot = d[leaving_idx]
            
            if abs(pivot) < self.tol:
                return SimplexResult(status=OptimizationStatus.NUMERICAL_ERROR)
            
            # Eta matrix update: more numerically stable
            eta_vector = d / pivot
            eta_vector[leaving_idx] = 1.0 / pivot
            
            # Update B_inv using rank-one update
            e_leaving = np.zeros(m)
            e_leaving[leaving_idx] = 1.0
            
            B_inv = B_inv - np.outer(B_inv @ A_entering, e_leaving @ B_inv) / pivot
            
            # Update basic variables
            basic_vars[leaving_idx] = entering_var
            
            iterations += 1
            
            if self.verbose and iterations % 10 == 0:
                current_obj = c @ (B_inv @ b if len(basic_vars) == m else np.zeros(n))
                print(f"Iteration {iterations}, objective estimate: {current_obj}")
        
        return SimplexResult(status=OptimizationStatus.MAX_ITERATIONS, iterations=iterations)
    
    def _btran(self, B_inv: np.ndarray, vector: np.ndarray) -> np.ndarray:
        """BTRAN operation: solve B^T * y = vector"""
        return B_inv.T @ vector
    
    def _ftran(self, B_inv: np.ndarray, vector: np.ndarray) -> np.ndarray:
        """FTRAN operation: solve B * x = vector, i.e., x = B^{-1} * vector"""
        return B_inv @ vector

def solve_lp_revised(c: np.ndarray, 
                    A: np.ndarray, 
                    b: np.ndarray,
                    pivot_rule: str = "dantzig",
                    verbose: bool = False) -> SimplexResult:
    """
    Convenience function to solve LP using revised simplex method.
    
    Args:
        c: Objective coefficients
        A: Constraint matrix  
        b: RHS vector
        pivot_rule: "dantzig", "bland", or "steepest_edge"
        verbose: Print iteration info
        
    Returns:
        SimplexResult with solution
    """
    pivot_rule_enum = {
        "dantzig": PivotRule.DANTZIG,
        "bland": PivotRule.BLAND, 
        "steepest_edge": PivotRule.STEEPEST_EDGE
    }[pivot_rule.lower()]
    
    solver = RevisedSimplexSolver(pivot_rule=pivot_rule_enum, verbose=verbose)
    return solver.solve(c, A, b) 

def solve_with_gurobi(c_orig: np.ndarray,
                     A_ineq: np.ndarray, 
                     b_ineq: np.ndarray,
                     bounds: Optional[List[Tuple[float, float]]] = None) -> Dict:
    """
    Solve LP with Gurobi for verification.
    
    Args:
        c_orig: Original objective coefficients (without slack vars)
        A_ineq: Inequality constraint matrix (Ax <= b form)
        b_ineq: Inequality RHS vector
        bounds: Variable bounds [(lb, ub), ...], default is [(0, inf), ...]
        
    Returns:
        Dictionary with Gurobi solution results
    """
    try:
        import gurobipy as gp
        from gurobipy import GRB
    except ImportError:
        print("Gurobi not available. Install with: pip install gurobipy")
        return {"status": "gurobi_not_available"}
    
    try:
        # Create model
        model = gp.Model("verification_lp")
        model.setParam('OutputFlag', 0)  # Suppress output
        
        n = len(c_orig)
        
        # Add variables
        if bounds is None:
            bounds = [(0, GRB.INFINITY)] * n
        
        vars = []
        for i in range(n):
            lb, ub = bounds[i]
            var = model.addVar(lb=lb, ub=ub, name=f"x{i}")
            vars.append(var)
        
        # Set objective (Gurobi minimizes by default)
        model.setObjective(sum(c_orig[i] * vars[i] for i in range(n)), GRB.MINIMIZE)
        
        # Add constraints
        for i in range(len(b_ineq)):
            model.addConstr(sum(A_ineq[i, j] * vars[j] for j in range(n)) <= b_ineq[i])
        
        # Solve
        model.optimize()
        
        # Extract results
        if model.status == GRB.OPTIMAL:
            x_gurobi = np.array([var.x for var in vars])
            obj_gurobi = model.objVal
            return {
                "status": "optimal",
                "x": x_gurobi,
                "objective_value": obj_gurobi,
                "gurobi_status": model.status
            }
        elif model.status == GRB.UNBOUNDED:
            return {"status": "unbounded", "gurobi_status": model.status}
        elif model.status == GRB.INFEASIBLE:
            return {"status": "infeasible", "gurobi_status": model.status}
        else:
            return {"status": "other", "gurobi_status": model.status, "gurobi_status_code": model.status}
            
    except Exception as e:
        return {"status": "error", "error": str(e)}

def compare_solutions(our_result: SimplexResult, gurobi_result: Dict, tolerance: float = 1e-6):
    """Compare our simplex solution with Gurobi's solution"""
    print("="*60)
    print("SOLUTION COMPARISON")
    print("="*60)
    
    print(f"Our Solver Status:    {our_result.status}")
    print(f"Gurobi Status:        {gurobi_result.get('status', 'N/A')}")
    
    if (our_result.status == OptimizationStatus.OPTIMAL and 
        gurobi_result.get('status') == 'optimal'):
        
        our_x = our_result.x[:len(gurobi_result['x'])]  # Only original variables
        gurobi_x = gurobi_result['x']
        
        print(f"\nOur Solution:         {our_x}")
        print(f"Gurobi Solution:      {gurobi_x}")
        print(f"Solution Difference:  {np.abs(our_x - gurobi_x)}")
        print(f"Max Difference:       {np.max(np.abs(our_x - gurobi_x))}")
        
        print(f"\nOur Objective:        {our_result.objective_value}")
        print(f"Gurobi Objective:     {gurobi_result['objective_value']}")
        print(f"Objective Difference: {abs(our_result.objective_value - gurobi_result['objective_value'])}")
        
        # Check if solutions match within tolerance
        x_match = np.allclose(our_x, gurobi_x, atol=tolerance)
        obj_match = abs(our_result.objective_value - gurobi_result['objective_value']) < tolerance
        
        print(f"\nSolutions Match:      {x_match}")
        print(f"Objectives Match:     {obj_match}")
        print(f"Overall Match:        {x_match and obj_match}")
        
        if x_match and obj_match:
            print("✓ VERIFICATION SUCCESSFUL!")
        else:
            print("✗ VERIFICATION FAILED!")
    else:
        print("Cannot compare - different solution statuses")

def run_comprehensive_tests():
    """Run comprehensive tests comparing our solver with Gurobi"""
    
    print("COMPREHENSIVE SIMPLEX SOLVER VERIFICATION")
    print("="*80)
    
    # Test Case 1: Standard 2D problem
    print("\n1. STANDARD 2D PROBLEM")
    print("min -3x1 - 2x2")
    print("s.t. x1 + x2 <= 4")
    print("     2x1 + x2 <= 6")
    print("     x1, x2 >= 0")
    
    # Original problem data (inequality form)
    c_orig = np.array([-3, -2])
    A_ineq = np.array([[1, 1], [2, 1]])
    b_ineq = np.array([4, 6])
    
    # Convert to standard form for our solver
    c_std = np.array([-3, -2, 0, 0])  # Add slack variables
    A_std = np.array([[1, 1, 1, 0], [2, 1, 0, 1]])  # Add identity for slacks
    b_std = np.array([4, 6])
    
    # Solve with our methods
    print("\n--- Our Primal Simplex ---")
    result_primal = solve_lp(c_std, A_std, b_std, method="primal", verbose=False)
    print(f"Status: {result_primal.status}")
    if result_primal.x is not None:
        print(f"Solution: x = {result_primal.x[:2]}")
        print(f"Objective: {result_primal.objective_value}")
        print(f"Iterations: {result_primal.iterations}")
    
    print("\n--- Our Dual Simplex ---")
    result_dual = solve_lp(c_std, A_std, b_std, method="dual", verbose=False)
    print(f"Status: {result_dual.status}")
    if result_dual.x is not None:
        print(f"Solution: x = {result_dual.x[:2]}")
        print(f"Objective: {result_dual.objective_value}")
        print(f"Iterations: {result_dual.iterations}")
    
    # Solve with Gurobi
    print("\n--- Gurobi Solution ---")
    gurobi_result = solve_with_gurobi(c_orig, A_ineq, b_ineq)
    if gurobi_result['status'] == 'optimal':
        print(f"Status: {gurobi_result['status']}")
        print(f"Solution: x = {gurobi_result['x']}")
        print(f"Objective: {gurobi_result['objective_value']}")
    else:
        print(f"Gurobi result: {gurobi_result}")
    
    # Compare results
    if result_primal.status == OptimizationStatus.OPTIMAL:
        compare_solutions(result_primal, gurobi_result)
    
    # Test Case 2: Unbounded problem
    print("\n\n2. UNBOUNDED PROBLEM")
    print("min -x1 - x2")
    print("s.t. -x1 + x2 <= 1")
    print("     x1, x2 >= 0")
    
    c_orig2 = np.array([-1, -1])
    A_ineq2 = np.array([[-1, 1]])
    b_ineq2 = np.array([1])
    
    c_std2 = np.array([-1, -1, 0])
    A_std2 = np.array([[-1, 1, 1]])
    b_std2 = np.array([1])
    
    print("\n--- Our Primal Simplex ---")
    result2 = solve_lp(c_std2, A_std2, b_std2, method="primal", verbose=False)
    print(f"Status: {result2.status}")
    
    print("\n--- Gurobi Solution ---")
    gurobi_result2 = solve_with_gurobi(c_orig2, A_ineq2, b_ineq2)
    print(f"Gurobi Status: {gurobi_result2['status']}")
    if 'gurobi_status_code' in gurobi_result2:
        print(f"Gurobi Status Code: {gurobi_result2['gurobi_status_code']}")
    
    # Note: Gurobi status 5 = INF_OR_UNBD, which often happens for unbounded problems
    gurobi_unbounded = (gurobi_result2['status'] == 'unbounded' or 
                       gurobi_result2.get('gurobi_status_code') == 5 or
                       gurobi_result2['status'] == 'other')
    print(f"\nBoth detect unbounded: {result2.status == OptimizationStatus.UNBOUNDED and gurobi_unbounded}")
    
    # Test Case 3: Infeasible problem
    print("\n\n3. INFEASIBLE PROBLEM")
    print("min x1 + x2")
    print("s.t. x1 + x2 <= -1")
    print("     x1, x2 >= 0")
    
    c_orig3 = np.array([1, 1])
    A_ineq3 = np.array([[1, 1]])
    b_ineq3 = np.array([-1])
    
    c_std3 = np.array([1, 1, 0])
    A_std3 = np.array([[1, 1, 1]])
    b_std3 = np.array([-1])
    
    print("\n--- Our Primal Simplex ---")
    result3 = solve_lp(c_std3, A_std3, b_std3, method="primal", verbose=False)
    print(f"Status: {result3.status}")
    
    print("\n--- Gurobi Solution ---")
    gurobi_result3 = solve_with_gurobi(c_orig3, A_ineq3, b_ineq3)
    print(f"Gurobi Status: {gurobi_result3['status']}")
    
    print(f"\nBoth detect infeasible: {result3.status == OptimizationStatus.INFEASIBLE and gurobi_result3['status'] == 'infeasible'}")

# Example usage and test cases
if __name__ == "__main__":
    run_comprehensive_tests()

COMPREHENSIVE SIMPLEX SOLVER VERIFICATION

1. STANDARD 2D PROBLEM
min -3x1 - 2x2
s.t. x1 + x2 <= 4
     2x1 + x2 <= 6
     x1, x2 >= 0

--- Our Primal Simplex ---
Status: OptimizationStatus.OPTIMAL
Solution: x = [2. 2.]
Objective: -10.0
Iterations: 2

--- Our Dual Simplex ---
Status: OptimizationStatus.OPTIMAL
Solution: x = [2. 2.]
Objective: -10.0
Iterations: 2

--- Gurobi Solution ---
Status: optimal
Solution: x = [2. 2.]
Objective: -10.0
SOLUTION COMPARISON
Our Solver Status:    OptimizationStatus.OPTIMAL
Gurobi Status:        optimal

Our Solution:         [2. 2.]
Gurobi Solution:      [2. 2.]
Solution Difference:  [0. 0.]
Max Difference:       0.0

Our Objective:        -10.0
Gurobi Objective:     -10.0
Objective Difference: 0.0

Solutions Match:      True
Objectives Match:     True
Overall Match:        True
✓ VERIFICATION SUCCESSFUL!


2. UNBOUNDED PROBLEM
min -x1 - x2
s.t. -x1 + x2 <= 1
     x1, x2 >= 0

--- Our Primal Simplex ---
Status: OptimizationStatus.UNBOUNDED

--- Gurob

In [3]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 37.6 MB/s eta 0:00:00 0:00:01
